<a href="https://colab.research.google.com/github/mcmendenhall/automatingseo/blob/main/%F0%9F%90%B8_SF_All_Links_CSV_Cleaner_%5BMake_a_Copy!%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  Automate the manual in_links clean up steps in Everett Sizemore's brilliant tutorial on:
#  How I Found Internal Linking Opportunities With Vector Embeddings: https://moz.com/blog/internal-linking-opportunities-with-vector-embeddings
#  Dev: Britney Muller

import pandas as pd
import numpy as np
from google.colab import files

def clean_link_dataset(df):
    """
    Clean and process link dataset according to specified rules.

    Parameters:
    df (pandas.DataFrame): Input DataFrame containing link data

    Returns:
    pandas.DataFrame: Cleaned DataFrame
    """
    # Make a copy to avoid modifying the original
    df = df.copy()

    print("Initial shape:", df.shape)

    # 1. Sort by Type and filter for Hyperlinks
    print("\nSorting by Type and filtering for Hyperlinks...")
    df = df.sort_values('Type')
    df = df[df['Type'] == 'Hyperlink'].drop('Type', axis=1)
    print("Shape after Type filtering:", df.shape)

    # 2. Sort by Status Code and filter for 200
    print("\nSorting by Status Code and filtering for 200 status...")
    df = df.sort_values('Status Code')
    df = df[df['Status Code'] == 200].drop(['Status Code', 'Status'], axis=1)
    print("Shape after Status filtering:", df.shape)

    # 3. Delete specified columns
    columns_to_drop = [
        'Size (Bytes)', 'Follow', 'Target', 'Rel',
        'Path Type', 'Link Path', 'Link Origin'
    ]
    print("\nRemoving unnecessary columns...")
    df = df.drop(columns_to_drop, axis=1)
    print("Remaining columns:", df.columns.tolist())

    # 4. Sort by Link Position
    print("\nSorting by Link Position...")
    df = df.sort_values('Link Position')

    # 5. Filter for Content and Aside in Link Position
    print("\nFiltering for Content and Aside positions...")
    df = df[df['Link Position'].isin(['Content', 'Aside'])]
    print("Shape after Link Position filtering:", df.shape)

    # 6. Sort by Source and clean Source URLs
    print("\nSorting and cleaning Source URLs...")
    df = df.sort_values('Source')
    def is_valid_page(url):
        invalid_patterns = [
            # ***MODIFY THE BELOW URL PATTERNS IF THERE ARE CERTAIN STRUCTURES YOU WANT TO INCLUDE/EXCLUDE***
            'category/', 'tag/', 'sitemap', 'search', '/home/', 'index'
        ]
        return not any(pattern in url.lower() for pattern in invalid_patterns)

    df = df[df['Source'].apply(is_valid_page)]
    print("Shape after Source URL cleaning:", df.shape)

    # 7. Sort by Destination and clean Destination URLs
    print("\nSorting and cleaning Destination URLs...")
    df = df.sort_values('Destination')
    df = df[df['Destination'].apply(is_valid_page)]
    print("Shape after Destination URL cleaning:", df.shape)

    # 8. Sort by Alt Text (Z to A) and process Alt Text
    print("\nSorting by Alt Text and processing...")
    df = df.sort_values('Alt Text', ascending=False)
    df.loc[df['Alt Text'].notna(), 'Anchor'] = df['Alt Text']
    df = df.drop('Alt Text', axis=1)

    # 9. Handle self-linking URLs
    print("\nProcessing self-linking URLs...")
    # Insert 'links to self' column between Destination and Anchor
    columns = df.columns.tolist()
    anchor_index = columns.index('Anchor')
    columns.insert(anchor_index, 'links to self')

    # Create the links to self column
    df['links to self'] = np.where(df['Source'] == df['Destination'], 'Match', 'No Match')

    # Sort by links to self column A-Z
    df = df.sort_values('links to self')

    # Remove matching rows and the column
    df = df[df['links to self'] != 'Match']
    df = df.drop('links to self', axis=1)
    print("Shape after removing self-links:", df.shape)

    # Remove Link Position column as it's no longer needed
    df = df.drop('Link Position', axis=1)

    # Final column ordering
    final_columns = ['Source', 'Destination', 'Anchor']
    df = df[final_columns]

    return df

# Main execution
print("Please upload your CSV file...")
uploaded = files.upload()

# Get the filename and read the CSV
file_name = list(uploaded.keys())[0]
print(f"\nReading file: {file_name}")

# Read the CSV and display initial information
df = pd.read_csv(file_name)
print("\nInitial DataFrame Info:")
print(df.info())
print("\nColumn names in your DataFrame:")
print(df.columns.tolist())

# Clean the dataset
print("\nStarting cleaning process...")
cleaned_df = clean_link_dataset(df)

# Display results
print("\nCleaning complete!")
print("\nFinal DataFrame Info:")
print(cleaned_df.info())
print("\nFirst few rows of cleaned dataset:")
print(cleaned_df.head())

# Save the cleaned dataset
output_filename = 'cleaned_' + file_name
cleaned_df.to_csv(output_filename, index=False)
print(f"\nCleaned data saved to {output_filename}")

# Provide option to download the cleaned file
print("\nDownloading cleaned file...")
files.download(output_filename)

Please upload your CSV file...


Saving all_inlinks.csv to all_inlinks.csv

Reading file: all_inlinks.csv

Initial DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8958 entries, 0 to 8957
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Type           8958 non-null   object
 1   Source         8958 non-null   object
 2   Destination    8958 non-null   object
 3   Size (Bytes)   8958 non-null   int64 
 4   Alt Text       4 non-null      object
 5   Anchor         8726 non-null   object
 6   Status Code    8958 non-null   int64 
 7   Status         4998 non-null   object
 8   Follow         8958 non-null   bool  
 9   Target         482 non-null    object
 10  Rel            485 non-null    object
 11  Path Type      8958 non-null   object
 12  Link Path      8928 non-null   object
 13  Link Position  8928 non-null   object
 14  Link Origin    8958 non-null   object
dtypes: bool(1), int64(2), object(12)
memory usage: 988.7+ KB
No

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>